<a href="https://colab.research.google.com/gist/ZeinHajjAli/774b2571b07362a799ff1dec58da99c9/buildingidentifiertestcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
!pip install -q keras

from glob import glob
from PIL import Image, ImageOps
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Model, load_model
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing import image as  kp_image
import os


Using TensorFlow backend.


In [0]:
!wget https://github.com/ZeinHajjAli/CarletonBuildingIdentifier/raw/master/test_folder.zip
!unzip test_folder.zip
!mv test_folder test
!wget https://github.com/ZeinHajjAli/CarletonBuildingIdentifier/raw/master/Models/Sequential.model

MODEL_FILE = "Sequential.model"

--2019-12-04 00:45:14--  https://github.com/ZeinHajjAli/CarletonBuildingIdentifier/raw/master/test_folder.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/ZeinHajjAli/CarletonBuildingIdentifier/master/test_folder.zip [following]
--2019-12-04 00:45:20--  https://raw.githubusercontent.com/ZeinHajjAli/CarletonBuildingIdentifier/master/test_folder.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44621124 (43M) [application/zip]
Saving to: ‘test_folder.zip’

test_folder.zip     100%[===================>]  42.55M   136MB/s    in 0.3s    

2019-12-04 00:45:23 (136 MB/s) - ‘test_folder.zip’ saved [4462

In [0]:
def prepareModel():
  global model # Sample global variable that you may populate here.
  model = load_model(MODEL_FILE)
  print("Model loaded.")

prepareModel()













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model loaded.


In [0]:
AA_test = glob('test/AA/*.jpg')
CB_test = glob('test/CB/*.jpg')
CT_test = glob('test/CT/*.jpg')
DT_test = glob('test/DT/*.jpg')
FH_test = glob('test/FH/*.jpg')
HP_test = glob('test/HP/*.jpg')
HS_test = glob('test/HS/*.jpg')
LB_test = glob('test/LB/*.jpg')
MC_test = glob('test/MC/*.jpg')
ME_test = glob('test/ME/*.jpg')
ML_test = glob('test/ML/*.jpg')
PA_test = glob('test/PA/*.jpg')
RB_test = glob('test/RB/*.jpg')
RO_test = glob('test/RO/*.jpg')
SA_test = glob('test/SA/*.jpg')
TB_test = glob('test/TB/*.jpg')

building_tests = (AA_test, CB_test, CT_test, DT_test, FH_test, HP_test, HS_test, LB_test, MC_test, ME_test, ML_test, PA_test, RB_test, RO_test, SA_test, TB_test)
labels = ('AA', 'CB', 'CT', 'DT', 'FH', 'HP', 'HS', 'LB', 'MC', 'ME', 'ML', 'PA', 'RB', 'RO', 'SA', 'TB')

In [0]:
preds = []
for i in range(16):
  buildingLabels = []
  for j in range(len(building_tests[i])):
    x = kp_image.img_to_array(Image.open(building_tests[i][j]).convert('RGB'))
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    pred = model.predict(x)
    maxIndex = 0
    for k in range(1, 16):
      if pred[0][k] > pred[0][maxIndex]:
        maxIndex = k
    # label = labels[maxIndex]
    label = maxIndex
    buildingLabels.append(label)
  preds.append(buildingLabels)

In [0]:
recalls = []
truePositives = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
falsePositives = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
for i in range(16):
  print('Recall for {}'.format(labels[i]))
  correct = 0
  for j in range(len(preds[i])):
    if preds[i][j] == i:
      truePositives[i] = truePositives[i]+1
    else:
      falsePositives[preds[i][j]] = falsePositives[preds[i][j]]+1
  rec = truePositives[i]/len(preds[i])
  recalls.append(rec)
  print(rec)

averageRecall = sum(recalls)/len(recalls)
print("The average recall is {}".format(averageRecall))
worstRecall = min(recalls)
print("The worst recall is that of {}, with a recall of {}".format(labels[recalls.index(worstRecall)], worstRecall))

Recall for AA
0.38095238095238093
Recall for CB
0.8571428571428571
Recall for CT
0.9534883720930233
Recall for DT
0.7619047619047619
Recall for FH
0.9767441860465116
Recall for HP
0.6829268292682927
Recall for HS
0.9523809523809523
Recall for LB
0.975609756097561
Recall for MC
0.6829268292682927
Recall for ME
0.7380952380952381
Recall for ML
0.6585365853658537
Recall for PA
0.7380952380952381
Recall for RB
0.7380952380952381
Recall for RO
0.6829268292682927
Recall for SA
0.8292682926829268
Recall for TB
0.8048780487804879
The average recall is 0.7758732747211193
The worst recall is that of AA, with a recall of 0.38095238095238093


In [0]:
precisions = []
for i in range(16):
  print('Precision for {}'.format(labels[i]))
  prec = truePositives[i] / (truePositives[i] + falsePositives[i])
  precisions.append(prec)
  print(prec)

averagePrecision = sum(precisions)/len(precisions)
print("The average precision is {}".format(averagePrecision))
bestPrecision = max(precisions)
print("The worst precision is that of {}, with a precision of {}".format(labels[precisions.index(bestPrecision)], bestPrecision))

Precision for AA
0.5925925925925926
Precision for CB
0.7058823529411765
Precision for CT
0.9761904761904762
Precision for DT
1.0
Precision for FH
0.9545454545454546
Precision for HP
0.4745762711864407
Precision for HS
0.8163265306122449
Precision for LB
0.975609756097561
Precision for MC
0.6511627906976745
Precision for ME
1.0
Precision for ML
0.574468085106383
Precision for PA
0.4626865671641791
Precision for RB
0.9393939393939394
Precision for RO
1.0
Precision for SA
0.8947368421052632
Precision for TB
0.9428571428571428
The average precision is 0.8100643000931581
The worst precision is that of DT, with a precision of 1.0


In [0]:
acc = sum(truePositives) / (sum(truePositives) + sum(falsePositives))
print(acc)

0.7766116941529235
